In [1]:
import os
import re
from pathlib import Path

import scanpy as sc
import pandas as pd
import numpy as np

In [7]:
exp = pd.read_csv('../data/bulk_rna_seq/cptac.txt', sep='\t', index_col=0)
exp

,C3L_00017,C3L_00102,C3L_00189,C3L_00277,C3L_00401,C3L_00589,C3L_00598,C3L_00599,C3L_00622,C3L_00625,...,C3N_03780,C3N_03839,C3N_03840,C3N_03853,C3N_03884,C3N_04119,C3N_04126,C3N_04282,C3N_04283,C3N_04284
gene,,,,,,,,,,,,,,,,,,,,,
5S_rRNA,0.01,0.00,0.02,0.00,0.00,0.02,0.04,0.00,0.04,0.04,...,0.00,0.00,0.01,0.01,0.00,0.00,0.02,0.00,0.00,0.00
5_8S_rRNA,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.15,0.00,0.00,...,0.02,0.00,0.00,0.00,0.04,0.00,0.00,0.00,0.00,0.00
7SK,5658.08,84.48,4786.91,276.88,6134.98,5941.98,321.65,6980.43,4642.36,6027.03,...,2919.75,5537.60,4204.96,371.82,3192.56,4564.49,4254.93,4823.87,123.61,705.33
A1BG,0.13,0.16,0.42,0.48,0.12,0.27,0.44,0.43,0.43,0.46,...,0.84,0.15,0.23,0.53,0.39,0.24,0.37,0.19,0.30,0.30
A1BG-AS1,2.48,2.34,6.04,2.45,3.35,2.45,1.53,3.90,4.58,4.85,...,4.63,2.39,3.30,1.64,3.12,4.87,4.25,3.03,2.66,2.12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZZEF1,101.47,97.85,107.22,136.63,129.20,99.41,102.02,117.65,118.63,111.62,...,123.25,72.71,127.62,117.91,89.28,97.34,101.99,88.63,68.30,166.06
ZZZ3,91.37,90.88,78.97,152.01,105.20,93.11,75.45,95.57,80.88,90.29,...,116.36,106.10,106.44,112.50,92.13,103.44,121.01,126.02,130.26,100.69
hsa-mir-1253,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [8]:
exp = exp.transpose()
exp

gene,5S_rRNA,5_8S_rRNA,7SK,A1BG,A1BG-AS1,A1CF,A2M,A2M-AS1,A2ML1,A2ML1-AS1,...,ZYG11A,ZYG11AP1,ZYG11B,ZYX,ZYXP1,ZZEF1,ZZZ3,hsa-mir-1253,hsa-mir-423,snoZ196
C3L_00017,0.01,0.0,5658.08,0.13,2.48,6.42,918.01,5.20,0.03,0.00,...,0.05,0.0,73.82,56.56,0.00,101.47,91.37,0.0,0.00,0.26
C3L_00102,0.00,0.0,84.48,0.16,2.34,14.27,1189.42,2.88,5.09,0.00,...,0.16,0.0,77.20,61.61,0.00,97.85,90.88,0.0,0.00,0.08
C3L_00189,0.02,0.0,4786.91,0.42,6.04,4.11,1061.75,6.32,19.72,0.00,...,0.22,0.0,88.17,92.45,0.25,107.22,78.97,0.0,0.00,0.14
C3L_00277,0.00,0.0,276.88,0.48,2.45,1.48,588.52,2.57,1.82,0.07,...,0.00,0.0,85.07,56.99,0.00,136.63,152.01,0.0,0.00,0.05
C3L_00401,0.00,0.0,6134.98,0.12,3.35,5.97,987.23,12.06,7.46,0.00,...,0.46,0.0,90.04,75.06,0.00,129.20,105.20,0.0,0.15,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C3N_04119,0.00,0.0,4564.49,0.24,4.87,4.49,986.17,2.80,21.31,0.05,...,0.11,0.0,94.29,53.47,0.00,97.34,103.44,0.0,0.00,0.05
C3N_04126,0.02,0.0,4254.93,0.37,4.25,12.01,913.32,4.68,39.72,0.00,...,1.04,0.0,82.98,52.61,0.00,101.99,121.01,0.0,0.00,0.05
C3N_04282,0.00,0.0,4823.87,0.19,3.03,1.42,720.18,2.48,20.63,0.00,...,0.12,0.0,96.46,63.82,0.00,88.63,126.02,0.0,0.00,0.00
C3N_04283,0.00,0.0,123.61,0.30,2.66,2.38,390.14,1.75,25.16,0.00,...,0.18,0.0,109.99,43.49,0.00,68.30,130.26,0.0,0.00,0.18


###### CYT score

The CYT score27 was computed as the geometric mean of GZMA and
PRF1 (as expressed in TPM, 0.01 offset)

Rooney, M. S., Shukla, S. A., Wu, C. J., Getz, G. & Hacohen, N. Molecular and genetic
properties of tumors associated with local immune cytolytic activity. Cell 160, 48–61
(2015).

In [20]:
cyt = exp[['GZMA', 'PRF1']].mean(axis=1)
cyt = pd.DataFrame(cyt / cyt.max())
cyt.columns = ['CYT score']
cyt

,CYT score
C3L_00017,0.112142
C3L_00102,0.091062
C3L_00189,0.117201
C3L_00277,0.053710
C3L_00401,0.176391
...,...
C3N_04119,0.050422
C3N_04126,0.164418
C3N_04282,0.042833
C3N_04283,0.133052


###### MCPcounter

This repository hosts the source code corresponding to the method described in our 2016 paper published in Genome Biology, Estimating the population abundance of tissue-infiltrating immune and stromal cell populations using gene expression

https://genomebiology.biomedcentral.com/articles/10.1186/s13059-016-1070-5

In [16]:
df = pd.read_csv('../data/immune_gene_sets/MCPcounter.txt', sep='\t')
df

,HUGO symbols,Cell population,ENTREZID,ENSEMBL ID
0,CD28,T cells,940,ENSG00000178562
1,CD3D,T cells,915,ENSG00000167286
2,CD3G,T cells,917,ENSG00000160654
3,CD5,T cells,921,ENSG00000110448
4,CD6,T cells,923,ENSG00000013725
...,...,...,...,...
106,COL6A2,Fibroblasts,1292,ENSG00000142173
107,DCN,Fibroblasts,1634,ENSG00000011465
108,GREM1,Fibroblasts,26585,ENSG00000166923
109,PAMR1,Fibroblasts,25891,ENSG00000149090


In [21]:
data = {}
for pop in sorted(set(df['Cell population'])):
    genes = df[df['Cell population']==pop]['HUGO symbols']
    f_exp = exp[[g for g in genes if g in exp.columns]].mean(axis=1)
    score = f_exp / f_exp.max()
    data[pop] = score
mcp = pd.DataFrame.from_dict(data)
mcp.index = exp.index.to_list()
mcp.columns = [f'MCPcounter {c}' for c in mcp.columns]
mcp

,MCPcounter B lineage,MCPcounter CD8 T cells,MCPcounter Cytotoxic lymphocytes,MCPcounter Endothelial cells,MCPcounter Fibroblasts,MCPcounter Monocytic lineage,MCPcounter Myeloid dendritic cells,MCPcounter NK cells,MCPcounter Neutrophils,MCPcounter T cells
C3L_00017,0.321187,0.108970,0.126219,0.620969,0.158521,0.425171,0.393011,0.291008,0.343533,0.215271
C3L_00102,0.078712,0.111981,0.137905,0.598698,0.318327,0.277237,0.366193,0.121047,0.461972,0.197549
C3L_00189,0.184587,0.148104,0.156719,0.619950,0.904329,0.366051,0.523039,0.280632,0.447074,0.169723
C3L_00277,0.055286,0.054184,0.064934,0.667925,0.202683,0.256895,0.093620,0.186759,0.399256,0.073540
C3L_00401,0.201150,0.214329,0.191779,0.441727,0.540858,0.600393,0.428362,0.603261,0.484306,0.274211
...,...,...,...,...,...,...,...,...,...,...
C3N_04119,0.018543,0.047562,0.053190,0.392855,0.813956,0.468867,0.242178,0.149209,0.426494,0.105104
C3N_04126,0.243245,0.226370,0.176672,0.569718,0.388789,0.601972,0.297684,0.192194,0.409744,0.214514
C3N_04282,0.029081,0.055388,0.049712,0.469824,0.665749,0.358898,0.256156,0.077569,0.311255,0.052505
C3N_04283,0.010644,0.082480,0.118123,0.426806,0.449206,0.262360,0.195124,0.155632,0.644300,0.094503


###### immunophenogram



z-score scaling of cancer immunity parameters30 to classify four different immune processes (MHC molecules,
immunomodulators, effector cells and suppressor cells), by generating
z-score normalised TPM gene expression for an input list of 162 genes.
Heatmaps used to visualise the data were generated using the pheatmap
R package (version 1.0.12) and unsupervised column hierarchical clustering based on the Euclidean distance performed

https://www.sciencedirect.com/science/article/pii/S2211124716317090?via%3Dihub#app2

In [22]:
df = pd.read_csv('../data/immune_gene_sets/immunophenogram.txt', sep='\t')
df

,Metagene,Cell type,Immunity
0,ADAM28,Activated B cell,Adaptive
1,CD180,Activated B cell,Adaptive
2,CD79B,Activated B cell,Adaptive
3,BLK,Activated B cell,Adaptive
4,CD19,Activated B cell,Adaptive
...,...,...,...
777,EMP3,Plasmacytoid dendritic cell,Innate
778,CD300LF,Plasmacytoid dendritic cell,Innate
779,ABTB1,Plasmacytoid dendritic cell,Innate
780,KLHL21,Plasmacytoid dendritic cell,Innate


In [25]:
pop_to_gene = {}
for ct, t, g in zip(df['Cell type'], df['Immunity'], df['Metagene']):
    key = f'{ct}: {t}'
    if key not in pop_to_gene:
        pop_to_gene[key] = []
    pop_to_gene[key].append(g)

pop_to_gene

{'Activated B cell: Adaptive': ['ADAM28',
  'CD180',
  'CD79B',
  'BLK',
  'CD19',
  'MS4A1',
  'TNFRSF17',
  'IGHM',
  'GNG7',
  'MICAL3',
  'SPIB',
  'HLA-DOB',
  'IGKC',
  'PNOC',
  'FCRL2',
  'BACH2',
  'CR2',
  'TCL1A',
  'AKNA',
  'ARHGAP25',
  'CCL21',
  'CD27',
  'CD38',
  'CLEC17A',
  'CLEC9A',
  'CLECL1'],
 'Activated CD4 T cell: Adaptive': ['AIM2',
  'BIRC3',
  'BRIP1',
  'CCL20',
  'CCL4',
  'CCL5',
  'CCNB1',
  'CCR7',
  'DUSP2',
  'ESCO2',
  'ETS1',
  'EXO1',
  'EXOC6',
  'IARS',
  'ITK',
  'KIF11',
  'KNTC1',
  'NUF2',
  'PRC1',
  'PSAT1',
  'RGS1',
  'RTKN2',
  'SAMSN1',
  'SELL',
  'TRAT1'],
 'Activated CD8 T cell: Adaptive': ['ADRM1',
  'AHSA1',
  'C1GALT1C1',
  'CCT6B',
  'CD37',
  'CD3D',
  'CD3E',
  'CD3G',
  'CD69',
  'CD8A',
  'CETN3',
  'CSE1L',
  'GEMIN6',
  'GNLY',
  'GPT2',
  'GZMA',
  'GZMH',
  'GZMK',
  'IL2RB',
  'LCK',
  'MPZL1',
  'NKG7',
  'PIK3IP1',
  'PTRH2',
  'TIMM13',
  'ZAP70'],
 'Central memory CD4 T cell: Adaptive': ['ABHD3',
  'AHNAK',
  'ANXA2

In [26]:
d = {}
for pop, genes in pop_to_gene.items():
    f_exp = exp[[g for g in genes if g in exp.columns]].mean(axis=1)
    score = f_exp / f_exp.max()
    d[pop] = score
iph = pd.DataFrame.from_dict(d)
iph.index = exp.index.to_list()
iph.columns = [f'Immunophenogram {c}' for c in iph.columns]
iph

,Immunophenogram Activated B cell: Adaptive,Immunophenogram Activated CD4 T cell: Adaptive,Immunophenogram Activated CD8 T cell: Adaptive,Immunophenogram Central memory CD4 T cell: Adaptive,Immunophenogram Central memory CD8 T cell: Adaptive,Immunophenogram Effector memeory CD4 T cell: Adaptive,Immunophenogram Effector memeory CD8 T cell: Adaptive,Immunophenogram Gamma delta T cell: Adaptive,Immunophenogram Immature B cell: Adaptive,Immunophenogram Memory B cell: Adaptive,...,Immunophenogram Eosinophil: Innate,Immunophenogram Immature dendritic cell: Innate,Immunophenogram Macrophage: Innate,Immunophenogram Mast cell: Innate,Immunophenogram MDSC: Innate,Immunophenogram Monocyte: Innate,Immunophenogram Natural killer cell: Innate,Immunophenogram Natural killer T cell: Innate,Immunophenogram Neutrophil: Innate,Immunophenogram Plasmacytoid dendritic cell: Innate
C3L_00017,0.256139,0.491636,0.640462,0.586740,0.611359,0.637152,0.648732,0.664573,0.493745,0.527082,...,0.200709,0.615509,0.247135,0.437684,0.466055,0.664615,0.159002,0.314896,0.475478,0.663182
C3L_00102,0.149099,0.476797,0.571027,0.812351,0.699770,0.656952,0.531137,0.683552,0.361689,0.589469,...,0.410548,0.776042,0.287415,0.432575,0.430631,0.848280,0.349393,0.553463,0.486641,0.798041
C3L_00189,0.159300,0.437171,0.541832,0.866183,0.630361,0.555443,0.573632,0.703410,0.395483,0.515101,...,0.283143,0.707287,0.290943,0.414978,0.410041,0.696430,0.684784,0.418451,0.524810,0.833560
C3L_00277,0.101913,0.272850,0.525157,0.919617,0.671693,0.623702,0.578443,0.729880,0.618340,0.619111,...,0.135721,0.648283,0.268955,0.342814,0.266503,0.743634,0.521321,0.499150,0.404638,0.753071
C3L_00401,0.257295,0.490972,0.576939,0.784532,0.616883,0.710800,0.689965,0.739769,0.436165,0.602289,...,0.230712,0.802718,0.389209,0.593683,0.502414,0.670755,0.399886,0.638935,0.598963,0.828780
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C3N_04119,0.045655,0.315552,0.458185,0.901125,0.785662,0.592896,0.475130,0.740921,0.292878,0.572520,...,0.143618,0.720775,0.259885,0.520155,0.417765,0.751650,0.684461,0.636027,0.601820,0.818223
C3N_04126,0.220357,0.538384,0.568738,0.686352,0.717543,0.768021,0.601442,0.653822,0.490191,0.551391,...,0.344499,0.729162,0.294011,0.530920,0.483988,0.674167,0.350476,0.469219,0.515448,0.825204
C3N_04282,0.050329,0.324863,0.458483,0.999505,0.716576,0.597674,0.400740,0.705867,0.521568,0.464541,...,0.157098,0.882659,0.274295,0.780823,0.402978,0.890878,0.498421,0.699606,0.349137,0.772918
C3N_04283,0.038575,0.451527,0.513781,1.000000,0.604177,0.525427,0.496002,0.775412,0.421734,0.391920,...,0.162402,0.830823,0.244349,0.387190,0.336257,0.875782,0.405316,0.543607,0.486185,0.840549


In [27]:
combined = pd.merge(cyt, mcp, right_index=True, left_index=True)
combined = pd.merge(combined, iph, right_index=True, left_index=True)
combined

,CYT score,MCPcounter B lineage,MCPcounter CD8 T cells,MCPcounter Cytotoxic lymphocytes,MCPcounter Endothelial cells,MCPcounter Fibroblasts,MCPcounter Monocytic lineage,MCPcounter Myeloid dendritic cells,MCPcounter NK cells,MCPcounter Neutrophils,...,Immunophenogram Eosinophil: Innate,Immunophenogram Immature dendritic cell: Innate,Immunophenogram Macrophage: Innate,Immunophenogram Mast cell: Innate,Immunophenogram MDSC: Innate,Immunophenogram Monocyte: Innate,Immunophenogram Natural killer cell: Innate,Immunophenogram Natural killer T cell: Innate,Immunophenogram Neutrophil: Innate,Immunophenogram Plasmacytoid dendritic cell: Innate
C3L_00017,0.112142,0.321187,0.108970,0.126219,0.620969,0.158521,0.425171,0.393011,0.291008,0.343533,...,0.200709,0.615509,0.247135,0.437684,0.466055,0.664615,0.159002,0.314896,0.475478,0.663182
C3L_00102,0.091062,0.078712,0.111981,0.137905,0.598698,0.318327,0.277237,0.366193,0.121047,0.461972,...,0.410548,0.776042,0.287415,0.432575,0.430631,0.848280,0.349393,0.553463,0.486641,0.798041
C3L_00189,0.117201,0.184587,0.148104,0.156719,0.619950,0.904329,0.366051,0.523039,0.280632,0.447074,...,0.283143,0.707287,0.290943,0.414978,0.410041,0.696430,0.684784,0.418451,0.524810,0.833560
C3L_00277,0.053710,0.055286,0.054184,0.064934,0.667925,0.202683,0.256895,0.093620,0.186759,0.399256,...,0.135721,0.648283,0.268955,0.342814,0.266503,0.743634,0.521321,0.499150,0.404638,0.753071
C3L_00401,0.176391,0.201150,0.214329,0.191779,0.441727,0.540858,0.600393,0.428362,0.603261,0.484306,...,0.230712,0.802718,0.389209,0.593683,0.502414,0.670755,0.399886,0.638935,0.598963,0.828780
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C3N_04119,0.050422,0.018543,0.047562,0.053190,0.392855,0.813956,0.468867,0.242178,0.149209,0.426494,...,0.143618,0.720775,0.259885,0.520155,0.417765,0.751650,0.684461,0.636027,0.601820,0.818223
C3N_04126,0.164418,0.243245,0.226370,0.176672,0.569718,0.388789,0.601972,0.297684,0.192194,0.409744,...,0.344499,0.729162,0.294011,0.530920,0.483988,0.674167,0.350476,0.469219,0.515448,0.825204
C3N_04282,0.042833,0.029081,0.055388,0.049712,0.469824,0.665749,0.358898,0.256156,0.077569,0.311255,...,0.157098,0.882659,0.274295,0.780823,0.402978,0.890878,0.498421,0.699606,0.349137,0.772918
C3N_04283,0.133052,0.010644,0.082480,0.118123,0.426806,0.449206,0.262360,0.195124,0.155632,0.644300,...,0.162402,0.830823,0.244349,0.387190,0.336257,0.875782,0.405316,0.543607,0.486185,0.840549


In [28]:
combined.to_csv('../data/cptac/immune_scores.txt', sep='\t')

In [29]:
gene_sets = {
    'CYT_score_genes': ['GZMA', 'PRF1']
}
for pop, genes in pop_to_gene.items():
    gene_sets[f'Immunophenogram_{pop}'] = genes

df = pd.read_csv('../data/immune_gene_sets/MCPcounter.txt', sep='\t')
for pop in sorted(set(df['Cell population'])):
    genes = df[df['Cell population']==pop]['HUGO symbols']
    gene_sets[f'MCPcounter_{pop}'] = genes.to_list()
gene_sets


{'CYT_score_genes': ['GZMA', 'PRF1'],
 'Immunophenogram_Activated B cell: Adaptive': ['ADAM28',
  'CD180',
  'CD79B',
  'BLK',
  'CD19',
  'MS4A1',
  'TNFRSF17',
  'IGHM',
  'GNG7',
  'MICAL3',
  'SPIB',
  'HLA-DOB',
  'IGKC',
  'PNOC',
  'FCRL2',
  'BACH2',
  'CR2',
  'TCL1A',
  'AKNA',
  'ARHGAP25',
  'CCL21',
  'CD27',
  'CD38',
  'CLEC17A',
  'CLEC9A',
  'CLECL1'],
 'Immunophenogram_Activated CD4 T cell: Adaptive': ['AIM2',
  'BIRC3',
  'BRIP1',
  'CCL20',
  'CCL4',
  'CCL5',
  'CCNB1',
  'CCR7',
  'DUSP2',
  'ESCO2',
  'ETS1',
  'EXO1',
  'EXOC6',
  'IARS',
  'ITK',
  'KIF11',
  'KNTC1',
  'NUF2',
  'PRC1',
  'PSAT1',
  'RGS1',
  'RTKN2',
  'SAMSN1',
  'SELL',
  'TRAT1'],
 'Immunophenogram_Activated CD8 T cell: Adaptive': ['ADRM1',
  'AHSA1',
  'C1GALT1C1',
  'CCT6B',
  'CD37',
  'CD3D',
  'CD3E',
  'CD3G',
  'CD69',
  'CD8A',
  'CETN3',
  'CSE1L',
  'GEMIN6',
  'GNLY',
  'GPT2',
  'GZMA',
  'GZMH',
  'GZMK',
  'IL2RB',
  'LCK',
  'MPZL1',
  'NKG7',
  'PIK3IP1',
  'PTRH2',
  'TIMM

In [30]:
import json

In [31]:
json.dump(gene_sets, open('../data/immune_gene_sets/all_gene_sets.json', 'w'))

In [2]:
pc = pd.read_csv('../data/protein_coding_genes/protein_coding_genes.txt', sep='\t')
pc

,gene,entrezgene_id,ensembl_gene_id,gene_biotype
0,A1BG,1.0,ENSG00000121410,protein_coding
1,A1BG-AS1,503538.0,ENSG00000268895,lncRNA
2,A1CF,29974.0,ENSG00000148584,protein_coding
3,A2M,2.0,ENSG00000175899,protein_coding
4,A2M-AS1,144571.0,ENSG00000245105,lncRNA
...,...,...,...,...
41941,ZYX,7791.0,ENSG00000159840,protein_coding
41942,ZYX,7791.0,ENSG00000285443,protein_coding
41943,ZYXP1,NaN,ENSG00000274572,unprocessed_pseudogene
41944,ZZEF1,23140.0,ENSG00000074755,protein_coding


In [4]:
f = pc[pc['gene_biotype']=='protein_coding']
f

,gene,entrezgene_id,ensembl_gene_id,gene_biotype
0,A1BG,1.0,ENSG00000121410,protein_coding
2,A1CF,29974.0,ENSG00000148584,protein_coding
3,A2M,2.0,ENSG00000175899,protein_coding
5,A2ML1,144568.0,ENSG00000166535,protein_coding
9,A3GALT2,127550.0,ENSG00000184389,protein_coding
...,...,...,...,...
41940,ZYG11B,79699.0,ENSG00000162378,protein_coding
41941,ZYX,7791.0,ENSG00000159840,protein_coding
41942,ZYX,7791.0,ENSG00000285443,protein_coding
41944,ZZEF1,23140.0,ENSG00000074755,protein_coding


In [5]:
[g for g in f['gene'] if 'RPS' in g]

['MRPS10',
 'MRPS11',
 'MRPS12',
 'MRPS12',
 'MRPS14',
 'MRPS15',
 'MRPS16',
 'MRPS17',
 'MRPS18A',
 'MRPS18B',
 'MRPS18B',
 'MRPS18B',
 'MRPS18B',
 'MRPS18B',
 'MRPS18B',
 'MRPS18B',
 'MRPS18C',
 'MRPS2',
 'MRPS21',
 'MRPS22',
 'MRPS23',
 'MRPS24',
 'MRPS25',
 'MRPS26',
 'MRPS27',
 'MRPS28',
 'MRPS30',
 'MRPS31',
 'MRPS33',
 'MRPS34',
 'MRPS35',
 'MRPS36',
 'MRPS36',
 'MRPS5',
 'MRPS6',
 'MRPS7',
 'MRPS9',
 'PRPS1',
 'PRPS1L1',
 'PRPS2',
 'PRPSAP1',
 'PRPSAP2',
 'RPS10',
 'RPS10-NUDT3',
 'RPS11',
 'RPS12',
 'RPS13',
 'RPS14',
 'RPS15',
 'RPS15A',
 'RPS16',
 'RPS17',
 'RPS17',
 'RPS18',
 'RPS18',
 'RPS18',
 'RPS18',
 'RPS18',
 'RPS18',
 'RPS19',
 'RPS19BP1',
 'RPS2',
 'RPS20',
 'RPS21',
 'RPS23',
 'RPS24',
 'RPS25',
 'RPS25',
 'RPS26',
 'RPS27',
 'RPS27A',
 'RPS27L',
 'RPS28',
 'RPS29',
 'RPS3',
 'RPS3A',
 'RPS4X',
 'RPS4Y1',
 'RPS4Y2',
 'RPS5',
 'RPS6',
 'RPS6KA1',
 'RPS6KA1',
 'RPS6KA2',
 'RPS6KA3',
 'RPS6KA4',
 'RPS6KA5',
 'RPS6KA6',
 'RPS6KB1',
 'RPS6KB2',
 'RPS6KC1',
 'RPS6KL1',
 